In [91]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [92]:
product_type = "ambient" #frozen chilled ambient 
num_time_periods = 365
storage_type="constrained" #standard constrained relaxed 
#Test in one article
test_article = "468a73f3"
M = 2000

In [93]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [94]:
# constants definitions
if(product_type=="frozen"):
    if(storage_type=="standard"):
        warehouse_volume = 30
    if(storage_type=="constrained"):
        warehouse_volume = 20
    if(storage_type=="relaxed"):
        warehouse_volume = 50
if(product_type=="chilled"):
    if(storage_type=="standard"):
        warehouse_volume = 250
    if(storage_type=="constrained"):
        warehouse_volume = 180
    if(storage_type=="relaxed"):
        warehouse_volume = 300
if(product_type=="ambient"):
    if(storage_type=="standard"):
        warehouse_volume = 700
    if(storage_type=="constrained"):
        warehouse_volume = 500
    if(storage_type=="relaxed"):
        warehouse_volume = 900
buffer_cost_per_m3 = 25
default_max_order = 10000
max_order_exceed_multiplier = 1.5

In [95]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [96]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [97]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [98]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [99]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [100]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [101]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}

        self.clearance_cost = 0
        self.buffer_cost = 0
        self.max_order_cost = 0
        self.fixed_cost = 0
        self.per_tu_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [102]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [103]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += demand[t][item]
                # cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    for item in cell.orders.keys():
        cell.orders[item] = math.ceil(cell.orders[item]/cu_per_tu[item])
        clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]

    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0

    fixed_cost = 0
    per_tu_cost = 0
    buffer_cost = 0
    max_order_cost = 0

    for item in cell.orders.keys():
        # Fixed and per tu cost for items
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        fixed_cost += ordering_cost_fixed[item]
        per_tu_cost += ordering_cost_per_tu[item] * cell.orders[item]

        # Max order penalty added
        if(cell.orders[item] > maximum_order_quantity_tu[item]):
            cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])
            max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])

        # Add costs for extra orders
        for extra_order_date in cell.extra_orders.keys():
            for extra_item in cell.extra_orders[extra_order_date].keys():
                cost += (ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]) + ordering_cost_fixed[extra_item]
                fixed_cost += ordering_cost_fixed[extra_item]
                per_tu_cost += ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]
                
                # Max order penalty for extra orders
                if(cell.extra_orders[extra_order_date][extra_item] > maximum_order_quantity_tu[extra_item]):
                    cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    
                    max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    

        # Keep track of volume
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    
    # Checking for extra volume and adding buffer cost
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3
            buffer_cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3

            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            # Removing volume of demand sold that day
            volume -= demand_volume

            # Adding volume of extra items from that day
            volume += fetchExtraVolume(cell, t)
        else:
            break

    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1)

    cell.buffer_cost = buffer_cost
    cell.max_order_cost = max_order_cost
    cell.fixed_cost = fixed_cost
    cell.per_tu_cost = per_tu_cost

order_date = 0
end_reached = False
obj_val = 0
schedule = {}

while(True):
    # Skipping ordering on sundays
    if(order_date%7==6):
        order_date += 1
        continue

    # Find date *until* which you want to order
    min_cost = float('inf')
    for t in range(order_date,n):
        # Skip checking demand for sundays
        if(t%7==6):
            continue

        schedule[order_date] = {}
        getCost(cost_matrix[order_date][t])

        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            # If looking at ordering until last day. Make the order
            if(t==n-1):
                schedule[order_date] = cost_matrix[order_date][t].orders
                end_reached = True
                obj_val += cost_matrix[order_date][t].cost

                cost_dict['buffer'] += cost_matrix[order_date][t].buffer_cost
                cost_dict['clearance'] += cost_matrix[order_date][t].clearance_cost
                cost_dict['fixed'] += cost_matrix[order_date][t].fixed_cost
                cost_dict['per_tu'] += cost_matrix[order_date][t].per_tu_cost
                cost_dict['max_order_penalty'] += cost_matrix[order_date][t].max_order_cost
                break
            
        # If cost increases, make order until previous day
        else:
            schedule[order_date] = cost_matrix[order_date][t-1].orders
            obj_val += cost_matrix[order_date][t-1].cost

            cost_dict['buffer'] += cost_matrix[order_date][t-1].buffer_cost
            cost_dict['clearance'] += cost_matrix[order_date][t-1].clearance_cost
            cost_dict['fixed'] += cost_matrix[order_date][t-1].fixed_cost
            cost_dict['per_tu'] += cost_matrix[order_date][t-1].per_tu_cost
            cost_dict['max_order_penalty'] += cost_matrix[order_date][t-1].max_order_cost

            # Place next order starting from today
            order_date = t
            break
    
    if(order_date >= n or end_reached):
        break

cost_dict['total'] = obj_val
cost_dict

{'fixed': 5002259.199999999,
 'per_tu': 24570593.99999992,
 'clearance': 1472689.7000000002,
 'buffer': 11317.1169032006,
 'max_order_penalty': 338987.55000000005,
 'total': 31395847.566903103}

In [104]:
result_dict = {}

for day, articles in schedule.items():
    for article, quantity in articles.items():
        if article in result_dict:
            result_dict[article][day] = quantity
        else:
            result_dict[article] = {day: quantity}
schedule = result_dict

In [105]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol

In [106]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = obj_val
    cost = obj_val
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule

    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            cost_improvement = 0
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i +1
                        
                    if order_days[day2] - order_days[day1] <= shelf_life[item]:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost_per_m3 * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            cost += delta
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            modified_orders[day1] = tot_orders
                            volumes[day1] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                modified_schedule[item] = modified_orders
            
            else: 
                if len(schedule[item]) == 1 :
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)
        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement

    return schedule, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [107]:
schedule, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta  = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

Buffer penalty paid 1211.2841515024998
max order penalty paid 18.299999999999997
fixed cost reduced by 32668.500000000055
31364408.651054595
Buffer penalty paid 1292.9852023026092
max order penalty paid 18.299999999999997
fixed cost reduced by 32750.500000000055
31364408.352105398
Buffer penalty paid 1373.136372502723
max order penalty paid 18.299999999999997
fixed cost reduced by 32832.50000000006
31364406.5032756
Buffer penalty paid 1453.2875427028364
max order penalty paid 18.299999999999997
fixed cost reduced by 32914.50000000006
31364404.6544458
Buffer penalty paid 1533.4387129029499
max order penalty paid 18.299999999999997
fixed cost reduced by 32996.50000000006
31364402.805616003
Buffer penalty paid 1613.5898831030634
max order penalty paid 18.299999999999997
fixed cost reduced by 33078.50000000006
31364400.956786204
Buffer penalty paid 1693.7410533031768
max order penalty paid 18.299999999999997
fixed cost reduced by 33160.50000000006
31364399.107956406
Buffer penalty paid 177

{'fixed': 4967766.199999999,
 'per_tu': 24570593.99999992,
 'clearance': 1472689.7000000002,
 'buffer': 14324.551106605591,
 'max_order_penalty': 339005.85000000003,
 'total': 31364380.301106542}

In [108]:
backup_schedule = schedule.copy()

In [109]:
schedule = backup_schedule.copy()

In [110]:
for item in schedule.keys():
    for day in schedule[item].keys():
        if(schedule[item][day] < 0):
            print("CRAP!")

In [111]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

In [112]:
cu_per_tu['59e744cc']

11

In [113]:
summ = 0
for day in demand.keys():
    if('59e744cc' in demand[day].keys()):
        summ += demand[day]['59e744cc']
        print(day, demand[day]['59e744cc'])
summ

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 0.0
64 0.0
65 0.0
66 0.0
67 0.0
68 0.0
69 0.0
70 0.0
71 20.0
72 26.0
73 47.0
74 42.0
75 22.0
76 0.0
77 29.0
78 15.0
79 35.0
80 48.0
81 32.0
82 20.0
83 0.0
84 34.0
85 18.0
86 36.0
87 43.0
88 32.0
89 23.0
90 0.0
91 25.0
92 19.0
93 23.0
94 43.0
95 29.0
96 20.0
97 0.0
98 35.0
99 24.0
100 40.0
101 34.0
102 0.0
103 61.0
104 0.0
105 0.0
106 37.0
107 36.0
108 42.0
109 38.0
110 13.0
111 0.0
112 46.0
113 33.0
114 45.0
115 33.0
116 51.0
117 10.0
118 0.0
119 20.0
120 22.0
121 15.0
122 51.0
123 52.0
124 13.0
125 0.0
126 28.0
127 20.0
128 22.0
129 30.0
130 45.0
131 10.0
132

12258.0

In [114]:
# test_schedule = schedule.copy()
# # test_schedule = transposeScheduleToDayFirst({'bd82f3b0':test_schedule['bd82f3b0']})
# test_schedule = transposeScheduleToDayFirst({'59e744cc':test_schedule['59e744cc']})
# articles = ['59e744cc']
# test_schedule

In [115]:
new_clearance_cost = 0
ordering_cost_per_tu_saved = 0

# Add clearance cost
# If I ever have to throw away equal to or more than a TU's worth, then I can just reduce order
def removeExpired(inv,life,item):
    global ordering_cost_per_tu_saved
    global new_clearance_cost
    
    tu_saved = 0
    if(len(inv)>life):
        print("CLEAR!")
        amount_cleared = inv[0]
        print("amount cleared: ", amount_cleared)

        tu_saved = amount_cleared//cu_per_tu[item]
        print("Ordering cost per tu saved: ", tu_saved * ordering_cost_per_tu[item])
        ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

        amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
        print("Inevitable clearance cost: ", amount_inevitably_cleared*clearing_cost_per_cu[item])
        new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]

        inv = inv[-life:]
    return inv, tu_saved

# Modify ordering costs
def addDay(inv,ordered):
    inv.append(ordered)
    return inv

def satisfyFromInventory(inv, dem):
    for day in range(len(inv)):
        if(dem==0):
            break
        if(inv[day] <= dem):
            dem -= inv[day]
            inv[day] = 0
        else:
            inv[day] -= dem
            dem = 0
    return inv, dem

def updateInventory(inv,item,t,ordered):
    life = shelf_life[item]
    dem = demand[t][item]
    inv = addDay(inv, ordered)
    inv, tu_saved = removeExpired(inv,life,item)
    print(inv)
    in_stock = sum(inv)
    extra_needed = (dem - in_stock) if (dem - in_stock) > 0 else 0
    if(extra_needed!=0):
        print("WOAH!", dem, in_stock)
        # print(inv)
    inv, remaining_dem = satisfyFromInventory(inv, dem)
    # print(inv, remaining_dem)
    # print(dem, ordered, in_stock, extra_needed)
    # inv = addDay(inv, ordered)
    # inv = removeExpired(inv,life)
    return inv, tu_saved

def calculateMaxOrderPenalty(schedule):
    max_order_penalty = 0
    for day in schedule.keys():
        for item in schedule[day].keys():
            if(schedule[day][item] > maximum_order_quantity_tu[item]):
                max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
    return max_order_penalty

def neighbourhoodOperator2(schedule):
    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    inventory = {}

    # schedule = transposeScheduleToDayFirst(schedule)
    for t in time_indexes:
        # incase no item was ordered on day t
        if t in schedule.keys():
            for item in articles:
                if item not in inventory.keys():
                    inventory[item] = []

                print("\ntime: ",t, item)
                if(t in schedule.keys() and item in schedule[t].keys()):
                    ordered = schedule[t][item] * cu_per_tu[item]
                else:
                    ordered = 0

                inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)
                if(t in schedule.keys() and item in schedule[t].keys()):
                    if(t>=shelf_life[item]):
                        if(schedule[t-shelf_life[item]][item] < tu_saved):
                            print("YO",schedule[t-shelf_life[item]][item],tu_saved)
                        schedule[t-shelf_life[item]][item] -= tu_saved

                
    print("Clearance cost: ", new_clearance_cost)
    print("Ordering cost per TU saved: ", ordering_cost_per_tu_saved)

    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    return schedule, new_clearance_cost, ordering_cost_per_tu_saved, new_max_order_penalty

In [116]:
schedule = transposeScheduleToDayFirst(schedule)
schedule, new_clearance_cost, per_tu_delta, new_max_order_penalty = neighbourhoodOperator2(schedule)
cost_dict['clearance'] = new_clearance_cost
cost_dict['max_order_penalty'] = new_max_order_penalty
cost_dict['per_tu'] -= per_tu_delta
cost_dict['total'] = cost_dict['fixed'] + cost_dict['per_tu'] + cost_dict['buffer'] + cost_dict['clearance'] + cost_dict['max_order_penalty']


time:  0 00027afc
[24]

time:  0 00084396
[15]

time:  0 0025253f
[40]

time:  0 00299fa6
[7]

time:  0 002b8d7d
[18]

time:  0 0034e982
[40]

time:  0 00415c43
[44]

time:  0 0057e9b7
[630]

time:  0 0059a703
[27]

time:  0 0062b624
[15]

time:  0 00718852
[35]

time:  0 0072bade
[77]

time:  0 007d0c15
[11]

time:  0 0098d2e1
[0]

time:  0 009aa26d
[40]

time:  0 00c31bc1
[0]

time:  0 00d09a16
[0]

time:  0 00e5e5d4
[42]

time:  0 00f7bc48
[26]

time:  0 00f95978
[50]

time:  0 010001ce
[12]

time:  0 01039c78
[54]

time:  0 0103bbf4
[0]

time:  0 0105db71
[0]

time:  0 010a9e81
[14]

time:  0 011f544e
[15]

time:  0 01527ae0
[30]

time:  0 01581e6a
[0]

time:  0 01594a1e
[0]

time:  0 015bf73f
[9]

time:  0 016d54fd
[0]

time:  0 0173b075
[18]

time:  0 018a81b7
[26]

time:  0 018afad9
[63]

time:  0 019b9022
[0]

time:  0 01b8b1c9
[0]

time:  0 01bfcb2c
[0]

time:  0 01c1f8cb
[8]

time:  0 01c4e30e
[24]

time:  0 01c5bf11
[30]

time:  0 01cac4fc
[15]

time:  0 01cb9c42
[45]

time

KeyError: 3

In [64]:
cost_dict

{'fixed': 111314.8999999999,
 'per_tu': 740290.199999998,
 'clearance': 0,
 'buffer': 307.88448304929403,
 'max_order_penalty': 7425.299999999999,
 'total': 859338.2844830472}